In [11]:
import pandas as pd
import inventorize3 as inv
import numpy as np
import seaborn as sns

# EOQ

In [14]:
d = 4000 #demand
p = 2500 #unit price
o = 6000 #ordering cost
h = 0.1 #holding cost in %

In [15]:
eoq = inv.eoq(d,o,p,h)

In [16]:
eoq = pd.DataFrame(eoq, index=[0])

In [21]:
eoq

,EOQ,T_years,T_weeks
0,438.178046,0.109545,5.696315


In [28]:
TLC = d/eoq.iloc[0,0]*o + eoq.iloc[0,0]/2*h 

In [29]:
TLC

54794.16465281682

In [25]:
eoq.iloc[0:,:]

0.10954451150103323

# -----------------------------------------------------------------------------------

# Safety Stock

## Preparing Data

In [12]:
df = pd.read_csv("clean_retail.csv")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797885 entries, 0 to 797884
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   797885 non-null  int64  
 1   Invoice      797885 non-null  object 
 2   StockCode    797885 non-null  object 
 3   Description  797885 non-null  object 
 4   Quantity     797885 non-null  int64  
 5   InvoiceDate  797885 non-null  object 
 6   Price        797885 non-null  float64
 7   Customer ID  797885 non-null  float64
 8   Country      797885 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 54.8+ MB


In [32]:
df

,Unnamed: 0,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Date
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009-12-01
1,1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01
2,2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009-12-01
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009-12-01
...,...,...,...,...,...,...,...,...,...,...
797880,1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-09
797881,1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09
797882,1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09
797883,1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2011-12-09


In [14]:
df["InvoiceDate"]= pd.to_datetime(df["InvoiceDate"])

In [15]:
df["InvoiceDate"]

0        2009-12-01 07:45:00
1        2009-12-01 07:45:00
2        2009-12-01 07:45:00
3        2009-12-01 07:45:00
4        2009-12-01 07:45:00
                 ...        
797880   2011-12-09 12:50:00
797881   2011-12-09 12:50:00
797882   2011-12-09 12:50:00
797883   2011-12-09 12:50:00
797884   2011-12-09 12:50:00
Name: InvoiceDate, Length: 797885, dtype: datetime64[ns]

In [16]:
df["Date"] = df["InvoiceDate"].dt.date

In [17]:
max_date = max(df["Date"])

In [33]:
max_date

Timestamp('2011-12-09 00:00:00')

## We'll use last 4 month data to get avg sales and std

In [34]:
LastFour = df[df["Date"] >= "2011-08-01"]

## Items with 0 sales are not present for that day so avg and stdev is wrong(as avg sales should be total sales/no. of days and since item is not present on all days we get a wrong avg same goes for stdev)

### there is a different way on chatgpt take a look for real world project

In [35]:
grp = LastFour.groupby(["Date","Description"]).agg(TotalSales = ("Quantity",np.sum)).reset_index()

C:\Users\75him\AppData\Local\Temp\ipykernel_23012\3440855913.py:1: FutureWarning: The provided callable <function sum at 0x0000020559694720> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  grp = LastFour.groupby(["Date","Description"]).agg(TotalSales = ("Quantity",np.sum)).reset_index()


In [36]:
grp

,Date,Description,TotalSales
0,2011-08-01,12 COLOURED PARTY BALLOONS,20
1,2011-08-01,12 EGG HOUSE PAINTED WOOD,1
2,2011-08-01,12 MESSAGE CARDS WITH ENVELOPES,12
3,2011-08-01,12 PENCILS SMALL TUBE SKULL,1
4,2011-08-01,12 PENCILS TALL TUBE SKULLS,12
...,...,...,...
102788,2011-12-09,ZINC SWEETHEART SOAP DISH,12
102789,2011-12-09,ZINC T-LIGHT HOLDER STAR LARGE,12
102790,2011-12-09,ZINC T-LIGHT HOLDER STARS SMALL,-11
102791,2011-12-09,ZINC WILLIE WINKIE CANDLE STICK,24


In [37]:
data = grp.groupby("Description").agg(AvgSales = ("TotalSales", np.mean), Stdev = ("TotalSales", np.std)).reset_index()

C:\Users\75him\AppData\Local\Temp\ipykernel_23012\3334511896.py:1: FutureWarning: The provided callable <function mean at 0x0000020559695800> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data = grp.groupby("Description").agg(AvgSales = ("TotalSales", np.mean), Stdev = ("TotalSales", np.std)).reset_index()
C:\Users\75him\AppData\Local\Temp\ipykernel_23012\3334511896.py:1: FutureWarning: The provided callable <function std at 0x0000020559695940> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  data = grp.groupby("Description").agg(AvgSales = ("TotalSales", np.mean), Stdev = ("TotalSales", np.std)).reset_index()


In [38]:
data

,Description,AvgSales,Stdev
0,4 PURPLE FLOCK DINNER CANDLES,5.300000,6.105218
1,50'S CHRISTMAS GIFT BAG LARGE,31.383333,55.507242
2,DOLLY GIRL BEAKER,29.743590,73.454012
3,I LOVE LONDON MINI BACKPACK,8.159091,11.966898
4,I LOVE LONDON MINI RUCKSACK,1.000000,NaN
...,...,...,...
3226,ZINC T-LIGHT HOLDER STARS SMALL,30.407407,37.576515
3227,ZINC TOP 2 DOOR WOODEN SHELF,1.000000,NaN
3228,ZINC WILLIE WINKIE CANDLE STICK,24.594203,30.790716
3229,ZINC WIRE KITCHEN ORGANISER,2.500000,2.828427


## Adding multicriteria ABC analysis to set service levels

In [39]:
LastFour

,Unnamed: 0,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Date
598439,810882,561904,22075,6 RIBBONS ELEGANT CHRISTMAS,96,2011-08-01 08:30:00,1.45,17941.0,United Kingdom,2011-08-01
598440,810883,561904,85049E,SCANDINAVIAN REDS RIBBONS,156,2011-08-01 08:30:00,1.06,17941.0,United Kingdom,2011-08-01
598441,810884,561905,21385,IVORY HANGING DECORATION HEART,24,2011-08-01 09:31:00,0.85,14947.0,United Kingdom,2011-08-01
598442,810885,561905,84970L,SINGLE HEART ZINC T-LIGHT HOLDER,12,2011-08-01 09:31:00,0.95,14947.0,United Kingdom,2011-08-01
598443,810886,561905,84970S,HANGING HEART ZINC T-LIGHT HOLDER,12,2011-08-01 09:31:00,0.85,14947.0,United Kingdom,2011-08-01
...,...,...,...,...,...,...,...,...,...,...
797880,1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-09
797881,1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09
797882,1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09
797883,1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2011-12-09


In [40]:
LastFour["Revenue"] = LastFour["Quantity"] * LastFour["Price"]

C:\Users\75him\AppData\Local\Temp\ipykernel_23012\960213401.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LastFour["Revenue"] = LastFour["Quantity"] * LastFour["Price"]


In [47]:
x = LastFour.groupby("Description").agg(TotalSales = ("Quantity",np.sum), TotalRevenue = ("Revenue",np.sum)).reset_index()

C:\Users\75him\AppData\Local\Temp\ipykernel_23012\1262200936.py:1: FutureWarning: The provided callable <function sum at 0x0000020559694720> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  x = LastFour.groupby("Description").agg(TotalSales = ("Quantity",np.sum), TotalRevenue = ("Revenue",np.sum)).reset_index()
C:\Users\75him\AppData\Local\Temp\ipykernel_23012\1262200936.py:1: FutureWarning: The provided callable <function sum at 0x0000020559694720> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  x = LastFour.groupby("Description").agg(TotalSales = ("Quantity",np.sum), TotalRevenue = ("Revenue",np.sum)).reset_index()


In [48]:
mabc = inv.productmix(x["Description"], x["TotalSales"], x["TotalRevenue"])

In [49]:
mabc

,skus,sales,revenue,sales_mix,comulative_sales,revenue_mix,comulative_revenue,sales_category,revenue_category,product_mix
2169,RABBIT NIGHT LIGHT,24112,45484.50,0.010212,0.022008,0.011411,0.011411,A,A,A_A
2270,REGENCY CAKESTAND 3 TIER,3493,40800.30,0.001479,0.332836,0.010236,0.021646,A,A,A_A
1438,JUMBO BAG RED RETROSPOT,20361,38459.93,0.008623,0.030631,0.009648,0.031295,A,A,A_A
1908,PAPER CHAIN KIT 50'S CHRISTMAS,12680,34788.68,0.005370,0.057538,0.008727,0.040022,A,A,A_A
3084,WHITE HANGING HEART T-LIGHT HOLDER,11694,32552.26,0.004953,0.072799,0.008166,0.048189,A,A,A_A
...,...,...,...,...,...,...,...,...,...,...
1152,GLASS BEURRE DISH,-8,-29.92,-0.000003,1.000129,-0.000008,1.002583,C,C,C_C
2963,VINTAGE BLUE KITCHEN CABINET,0,-75.00,0.000000,1.000134,-0.000019,1.002564,C,C,C_C
3102,WHITE TALL PORCELAIN T-LIGHT HOLDER,-208,-386.40,-0.000088,1.000000,-0.000097,1.002467,C,C,C_C
882,Discount,-37,-1899.52,-0.000016,1.000088,-0.000477,1.001990,C,C,C_C


In [51]:
mabc.rename(columns={"skus":"Description"}, inplace=True)

In [52]:
merged = pd.merge(data, mabc, on ="Description", how="inner")

In [53]:
merged

,Description,AvgSales,Stdev,sales,revenue,sales_mix,comulative_sales,revenue_mix,comulative_revenue,sales_category,revenue_category,product_mix
0,4 PURPLE FLOCK DINNER CANDLES,5.300000,6.105218,106,184.06,4.489359e-05,0.983503,0.000046,0.984258,C,C,C_C
1,50'S CHRISTMAS GIFT BAG LARGE,31.383333,55.507242,1883,2269.75,7.974965e-04,0.574856,0.000569,0.689379,A,A,A_A
2,DOLLY GIRL BEAKER,29.743590,73.454012,2320,2662.00,9.825766e-04,0.497322,0.000668,0.652449,A,A,A_A
3,I LOVE LONDON MINI BACKPACK,8.159091,11.966898,359,1449.85,1.520453e-04,0.921959,0.000364,0.793273,B,A,B_A
4,I LOVE LONDON MINI RUCKSACK,1.000000,NaN,1,4.15,4.235244e-07,1.000107,0.000001,1.002554,C,C,C_C
...,...,...,...,...,...,...,...,...,...,...,...,...
3226,ZINC T-LIGHT HOLDER STARS SMALL,30.407407,37.576515,2463,1987.65,1.043141e-03,0.478004,0.000499,0.729505,A,A,A_A
3227,ZINC TOP 2 DOOR WOODEN SHELF,1.000000,NaN,1,16.95,4.235244e-07,1.000108,0.000004,1.001935,C,C,C_C
3228,ZINC WILLIE WINKIE CANDLE STICK,24.594203,30.790716,1697,1403.45,7.187209e-04,0.610784,0.000352,0.799356,A,A,A_A
3229,ZINC WIRE KITCHEN ORGANISER,2.500000,2.828427,20,104.80,8.470488e-06,0.998365,0.000026,0.993847,C,C,C_C


In [54]:
lead_time = 12
std_lead_time = 2

In [55]:
merged.product_mix.value_counts()

product_mix
C_C    1465
A_A     500
B_B     314
B_C     237
C_B     235
A_B     228
B_A     175
A_C      42
C_A      35
Name: count, dtype: int64

In [56]:
mapping = {"A_A":0.95,"A_B":0.95,"A_C":0.95,"B_A":0.8,"B_B":0.7,"B_C":0.7,"C_A":0.8,"C_B":0.7,"C_C":0.6}

In [58]:
merged["ServiceLevel"] = merged["product_mix"].map(mapping)

In [60]:
from scipy.stats import norm

# Calculate Z-score from Service Level
merged["Z_Score"] = merged["ServiceLevel"].apply(norm.ppf)

In [61]:
merged

,Description,AvgSales,Stdev,sales,revenue,sales_mix,comulative_sales,revenue_mix,comulative_revenue,sales_category,revenue_category,product_mix,ServiceLevel,Z_Score
0,4 PURPLE FLOCK DINNER CANDLES,5.300000,6.105218,106,184.06,4.489359e-05,0.983503,0.000046,0.984258,C,C,C_C,0.60,0.253347
1,50'S CHRISTMAS GIFT BAG LARGE,31.383333,55.507242,1883,2269.75,7.974965e-04,0.574856,0.000569,0.689379,A,A,A_A,0.95,1.644854
2,DOLLY GIRL BEAKER,29.743590,73.454012,2320,2662.00,9.825766e-04,0.497322,0.000668,0.652449,A,A,A_A,0.95,1.644854
3,I LOVE LONDON MINI BACKPACK,8.159091,11.966898,359,1449.85,1.520453e-04,0.921959,0.000364,0.793273,B,A,B_A,0.80,0.841621
4,I LOVE LONDON MINI RUCKSACK,1.000000,NaN,1,4.15,4.235244e-07,1.000107,0.000001,1.002554,C,C,C_C,0.60,0.253347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3226,ZINC T-LIGHT HOLDER STARS SMALL,30.407407,37.576515,2463,1987.65,1.043141e-03,0.478004,0.000499,0.729505,A,A,A_A,0.95,1.644854
3227,ZINC TOP 2 DOOR WOODEN SHELF,1.000000,NaN,1,16.95,4.235244e-07,1.000108,0.000004,1.001935,C,C,C_C,0.60,0.253347
3228,ZINC WILLIE WINKIE CANDLE STICK,24.594203,30.790716,1697,1403.45,7.187209e-04,0.610784,0.000352,0.799356,A,A,A_A,0.95,1.644854
3229,ZINC WIRE KITCHEN ORGANISER,2.500000,2.828427,20,104.80,8.470488e-06,0.998365,0.000026,0.993847,C,C,C_C,0.60,0.253347


In [62]:
merged.drop(columns=["sales_mix","comulative_sales","revenue_mix","comulative_revenue",	"sales_category","revenue_category"], inplace=True)

In [63]:
merged

,Description,AvgSales,Stdev,sales,revenue,product_mix,ServiceLevel,Z_Score
0,4 PURPLE FLOCK DINNER CANDLES,5.300000,6.105218,106,184.06,C_C,0.60,0.253347
1,50'S CHRISTMAS GIFT BAG LARGE,31.383333,55.507242,1883,2269.75,A_A,0.95,1.644854
2,DOLLY GIRL BEAKER,29.743590,73.454012,2320,2662.00,A_A,0.95,1.644854
3,I LOVE LONDON MINI BACKPACK,8.159091,11.966898,359,1449.85,B_A,0.80,0.841621
4,I LOVE LONDON MINI RUCKSACK,1.000000,NaN,1,4.15,C_C,0.60,0.253347
...,...,...,...,...,...,...,...,...
3226,ZINC T-LIGHT HOLDER STARS SMALL,30.407407,37.576515,2463,1987.65,A_A,0.95,1.644854
3227,ZINC TOP 2 DOOR WOODEN SHELF,1.000000,NaN,1,16.95,C_C,0.60,0.253347
3228,ZINC WILLIE WINKIE CANDLE STICK,24.594203,30.790716,1697,1403.45,A_A,0.95,1.644854
3229,ZINC WIRE KITCHEN ORGANISER,2.500000,2.828427,20,104.80,C_C,0.60,0.253347


In [66]:
merged["ReorderQuantity"] = merged["AvgSales"]*lead_time

In [71]:
inv.saftey_stock_normal()

{'demandleadtime': 192.30769230769232,
 'sigmadl': 138.67504905630727,
 'safteyfactor': nan,
 'cyclestock+safteystock': nan}

In [72]:
merged["Safety_Stock"] = merged.apply(lambda row: inv.saftey_stock_normal(merged.sales,merged.Stdev,14, merged.Z_Score), axis=1)

In [77]:
merged.Safety_Stock.iloc[1]

{'demandleadtime': 0        28.538462
 1       506.961538
 2       624.615385
 3        96.653846
 4         0.269231
            ...    
 3226    663.115385
 3227      0.269231
 3228    456.884615
 3229      5.384615
 3230      8.884615
 Name: sales, Length: 3231, dtype: float64,
 'sigmadl': 0        3.167842
 1       28.801294
 2       38.113415
 3        6.209319
 4             NaN
           ...    
 3226    19.497496
 3227          NaN
 3228    15.976518
 3229     1.467599
 3230     2.002883
 Name: Stdev, Length: 3231, dtype: float64,
 'safteyfactor': array([-0.66399391,         nan,         nan, ...,         nan,
        -0.66399391, -0.66399391]),
 'cyclestock+safteystock': 0        26.435034
 1              NaN
 2              NaN
 3       102.870264
 4              NaN
            ...    
 3226           NaN
 3227           NaN
 3228           NaN
 3229      4.410139
 3230      7.554714
 Length: 3231, dtype: float64}

# ------------------------------------------------------------------------------------------

## Seasonal Inventory

In [4]:
mean = 50
sd = 10
c = 1.2
salvage = 0.7
penalty=0.4
price=3

In [2]:
import inventorize3 as inv

In [7]:
cr = inv.CriticalRatio(price, c, salvage, penalty)

In [8]:
cr

0.8148148148148149

In [9]:
inv.MPN_singleperiod(mean, sd, price,c,salvage,penalty)

{'quantity': 58.957798188835895,
 'demand': 50,
 'sd': 10,
 'unitshort': 1.012102766308183,
 'shortagecost': 2.732677469032094,
 'cost': 29.478899094417947,
 'revenue': 150,
 'profit': 82.78842343654995,
 'soldatfullprice': 48.98789723369182,
 'sold_discount': 9.969900955144077}

In [19]:
df

,Unnamed: 0,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Date
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009-12-01
1,1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01
2,2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009-12-01
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009-12-01
...,...,...,...,...,...,...,...,...,...,...
797880,1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-09
797881,1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09
797882,1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09
797883,1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2011-12-09


In [20]:
df["Year"]= df.InvoiceDate.dt.year

In [21]:
df

,Unnamed: 0,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Date,Year
0,0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009-12-01,2009
1,1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01,2009
2,2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01,2009
3,3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009-12-01,2009
4,4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009-12-01,2009
...,...,...,...,...,...,...,...,...,...,...,...
797880,1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-09,2011
797881,1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09,2011
797882,1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09,2011
797883,1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2011-12-09,2011


In [34]:
df2 = df[df["Year"].isin([2010,2011])]

In [35]:
df2

,Unnamed: 0,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Date,Year
31276,45228,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0,United Kingdom,2010-01-04,2010
31277,45229,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0,United Kingdom,2010-01-04,2010
31278,45230,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0,United Kingdom,2010-01-04,2010
31279,45234,493414,21844,RETRO SPOT MUG,36,2010-01-04 10:28:00,2.55,14590.0,United Kingdom,2010-01-04,2010
31280,45235,493414,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-04 10:28:00,4.25,14590.0,United Kingdom,2010-01-04,2010
...,...,...,...,...,...,...,...,...,...,...,...
797880,1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-09,2011
797881,1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09,2011
797882,1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-09,2011
797883,1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2011-12-09,2011


In [36]:
gp = df2.groupby(["Year","Description"]).agg(total_sales=("Quantity",np.sum),price=("Price",np.mean)).reset_index()

C:\Users\75him\AppData\Local\Temp\ipykernel_33640\3266853770.py:1: FutureWarning: The provided callable <function sum at 0x00000204EF4B5940> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  gp = df2.groupby(["Year","Description"]).agg(total_sales=("Quantity",np.sum),price=("Price",np.mean)).reset_index()
C:\Users\75him\AppData\Local\Temp\ipykernel_33640\3266853770.py:1: FutureWarning: The provided callable <function mean at 0x00000204EF4B6A20> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  gp = df2.groupby(["Year","Description"]).agg(total_sales=("Quantity",np.sum),price=("Price",np.mean)).reset_index()


In [37]:
gp

,Year,Description,total_sales,price
0,2010,DOORMAT UNION JACK GUNS AND ROSES,167,6.640196
1,2010,3 STRIPEY MICE FELTCRAFT,399,1.941429
2,2010,4 PURPLE FLOCK DINNER CANDLES,200,2.150000
3,2010,ANIMAL STICKERS,169,0.210000
4,2010,BLACK PIRATE TREASURE CHEST,6,1.650000
...,...,...,...,...
8137,2011,ZINC T-LIGHT HOLDER STARS SMALL,4850,0.836888
8138,2011,ZINC TOP 2 DOOR WOODEN SHELF,9,16.950000
8139,2011,ZINC WILLIE WINKIE CANDLE STICK,2420,0.873702
8140,2011,ZINC WIRE KITCHEN ORGANISER,25,7.175000


In [43]:
gb2 = gp.groupby(["Description"]).agg(avg_sales=("total_sales",np.mean), price = ("price",np.mean), std = ("total_sales",np.std)).reset_index()

C:\Users\75him\AppData\Local\Temp\ipykernel_33640\4267186090.py:1: FutureWarning: The provided callable <function mean at 0x00000204EF4B6A20> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  gb2 = gp.groupby(["Description"]).agg(avg_sales=("total_sales",np.mean), price = ("price",np.mean), std = ("total_sales",np.std)).reset_index()
C:\Users\75him\AppData\Local\Temp\ipykernel_33640\4267186090.py:1: FutureWarning: The provided callable <function std at 0x00000204EF4B6B60> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  gb2 = gp.groupby(["Description"]).agg(avg_sales=("total_sales",np.mean), price = ("price",np.mean), std = ("total_sales",np.std)).reset_index()
C:\Users\75him\AppData\Local\Temp\ipykernel_33640\4267186090.py:1: FutureWarning: T

In [44]:
gb2

,Description,avg_sales,price,std
0,DOORMAT UNION JACK GUNS AND ROSES,167.0,6.640196,NaN
1,3 STRIPEY MICE FELTCRAFT,399.0,1.941429,NaN
2,4 PURPLE FLOCK DINNER CANDLES,162.0,2.224286,53.740115
3,50'S CHRISTMAS GIFT BAG LARGE,1883.0,1.248091,NaN
4,ANIMAL STICKERS,169.0,0.210000,NaN
...,...,...,...,...
5166,ZINC T-LIGHT HOLDER STARS SMALL,4850.0,0.836888,NaN
5167,ZINC TOP 2 DOOR WOODEN SHELF,64.5,14.714706,78.488853
5168,ZINC WILLIE WINKIE CANDLE STICK,2812.0,0.860513,554.371716
5169,ZINC WIRE KITCHEN ORGANISER,25.0,7.175000,NaN


In [45]:
gb2["std"].isna().sum()

2200

In [47]:
gb2.loc[gb2['std'].isna(), 'std'] = gb2['avg_sales'] * 0.1

In [53]:
gb2["Cost"]= gb2.price*0.6

In [56]:
def apply_mpn(row):
    return pd.Series(
        inv.MPN_singleperiod(
            row['avg_sales'], row['std'], row['price'],row['Cost'],0,0
           )
    )

df_season = gb2.join(gb2.apply(apply_mpn, axis=1))

C:\Users\75him\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2246: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\75him\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2247: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [57]:
df_season

,Description,avg_sales,price,std,Cost,quantity,demand,sd,unitshort,shortagecost,cost,revenue,profit,soldatfullprice,sold_discount
0,DOORMAT UNION JACK GUNS AND ROSES,167.0,6.640196,16.700000,3.984118,162.769103,167.0,16.700000,8.990458,59.698406,648.491257,1108.912745,400.723082,158.009542,4.759562
1,3 STRIPEY MICE FELTCRAFT,399.0,1.941429,39.900000,1.164857,388.891451,399.0,39.900000,21.480197,41.702268,453.002984,774.630000,279.924748,377.519803,11.371647
2,4 PURPLE FLOCK DINNER CANDLES,162.0,2.224286,53.740115,1.334571,148.385097,162.0,53.740115,28.931034,64.350885,198.030511,360.334286,97.952889,133.068966,15.316131
3,50'S CHRISTMAS GIFT BAG LARGE,1883.0,1.248091,188.300000,0.748855,1835.294740,1883.0,188.300000,101.371455,126.520791,1374.368809,2350.155182,849.265582,1781.628545,53.666195
4,ANIMAL STICKERS,169.0,0.210000,16.900000,0.126000,164.718434,169.0,16.900000,9.098128,1.910607,20.754523,35.490000,12.824870,159.901872,4.816562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5166,ZINC T-LIGHT HOLDER STARS SMALL,4850.0,0.836888,485.000000,0.502133,4727.126655,4850.0,485.000000,261.100136,218.511562,2373.645249,4058.906639,1466.749828,4588.899864,138.226791
5167,ZINC TOP 2 DOOR WOODEN SHELF,64.5,14.714706,78.488853,8.828824,44.615077,64.5,78.488853,42.254536,621.763074,393.898637,949.098529,-66.563182,22.245464,22.369613
5168,ZINC WILLIE WINKIE CANDLE STICK,2812.0,0.860513,554.371716,0.516308,2671.551532,2812.0,554.371716,298.446454,256.817167,1379.343503,2419.763625,783.602954,2513.553546,157.997986
5169,ZINC WIRE KITCHEN ORGANISER,25.0,7.175000,2.500000,4.305000,24.366632,25.0,2.500000,1.345877,9.656667,104.898352,179.375000,64.819981,23.654123,0.712509
